<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_09_05_Monitor_Reddit_mentions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to ingest a subReddit and monitor for company mentions.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.


---

Install Graphlit Python client SDK

In [ ]:
!pip install --upgrade graphlit-client

Initialize Graphlit

In [ ]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Define Graphlit helper functions

In [ ]:
from typing import List, Optional

# Create entity extraction workflow, using Azure AI Text Analytics to identify organizations
async def create_workflow():
    if graphlit.client is None:
        return;

    input = input_types.WorkflowInput(
        name="Extraction",
        extraction=input_types.ExtractionWorkflowStageInput(
            jobs=[
                input_types.ExtractionWorkflowJobInput(
                    connector=input_types.EntityExtractionConnectorInput(
                        type=enums.EntityExtractionServiceTypes.AZURE_COGNITIVE_SERVICES_TEXT,
                        extractedTypes=[enums.ObservableTypes.ORGANIZATION]
                    )
                )
            ]
        )
    )

    try:
        response = await graphlit.client.create_workflow(input)

        return response.create_workflow.id if response.create_workflow is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_feed(name: str, workflow_id: str):
    if graphlit.client is None:
        return;

    input = input_types.FeedInput(
        name=name,
        type=enums.FeedTypes.REDDIT,
        reddit=input_types.RedditFeedPropertiesInput(
            subredditName=name,
            readLimit=50 # limiting to 50 Reddit posts
        ),
        workflow=input_types.EntityReferenceInput(
            id=workflow_id
        )
    )

    try:
        response = await graphlit.client.create_feed(input)

        return response.create_feed.id if response.create_feed is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def is_feed_done(feed_id: str):
    if graphlit.client is None:
        return;

    response = await graphlit.client.is_feed_done(feed_id)

    return response.is_feed_done.result if response.is_feed_done is not None else None

async def query_organizations(name: str):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.query_organizations(
            filter=input_types.OrganizationFilter(
                name=name
            )
        )

        return response.organizations.results if response.organizations is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

# Locate contents where organization was observed (by Azure AI Text Analytics)
async def query_contents(organization_id: str):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.query_contents(
            filter=input_types.ContentFilter(
                observations=[
                    input_types.ObservationReferenceFilter(
                        type=enums.ObservableTypes.ORGANIZATION,
                        observable=input_types.EntityReferenceFilter(
                            id=organization_id
                        )
                    )
                ]
            )
        )

        return response.contents.results if response.contents is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_all_workflows():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_workflows(is_synchronous=True)

async def delete_all_feeds():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_feeds(is_synchronous=True)


Execute Graphlit example

In [ ]:
from IPython.display import display, Markdown
import time

# NOTE: Fill in the Reddit subreddit name
subreddit_name = "Anthropic"

# NOTE: Fill in the organization name you're looking for
organization_name = "Google"

# Remove any existing feeds and workflows; only needed for notebook example
await delete_all_feeds()
await delete_all_workflows()

print('Deleted all feeds and workflows.')

workflow_id = await create_workflow()

if workflow_id is not None:
    print(f'Created workflow [{workflow_id}].')

    feed_id = await create_feed(name=subreddit_name, workflow_id=workflow_id)

    if feed_id is not None:
        print(f'Created feed [{feed_id}].')

        # Wait for feed to complete, since ingestion happens asychronously
        done = False
        time.sleep(5)
        while not done:
            done = await is_feed_done(feed_id)

            if not done:
                time.sleep(2)

        print(f'Completed feed [{feed_id}].')

        organizations = await query_organizations(organization_name)

        if organizations is not None and organizations.count != 0:
            for organization in organizations:
                if organization is not None:
                    print(f'Found organization [{organization.id}] named [{organization.name}].')

                    # Query contents by organization
                    contents = await query_contents(organization.id)

                    if contents is not None:
                        for content in contents:
                            if content is not None:
                                display(Markdown(f'### Found Reddit post [{content.id}] that mentioned [{organization_name}]'))
                                display(Markdown(content.markdown))
                                print()
        else:
            print(f'No organizations named [{organization_name}] found.')